In [1]:
!pip install pandas scikit-learn optuna seaborn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import os
from include.utils import incrementar_dados_aleatorios_csv, categorizar_imc
from include.utils import prever_encargos_futuros, segmentacao_de_risco, analise_de_sensibilidade, otimizacao_de_recursos, planejamento_estrategico

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

**1. Engenharia de Recursos**

Novos recursos derivados dos dados existentes

In [4]:
# Obtém a largura do terminal - uso mais embaixo nos prints
terminal_width = os.get_terminal_size().columns

dados = pd.read_csv('../planilhas/dados_aleatorios_sobre_original.csv', encoding='latin-1')

# Aplicar a função para categorizar IMC e criar uma nova coluna 'Categoria IMC'
dados['Categoria IMC'] = dados['IMC'].apply(categorizar_imc)

# Mapeamento das regiões para valores numéricos
regiao_numerica = {'sudoeste': 1, 'sudeste': 2, 'nordeste': 3, 'noroeste': 4}

# Convertendo os valores da coluna 'Região' para valores numéricos usando o mapeamento
dados['Região_numerica'] = dados['Região'].map(regiao_numerica)

**2. Limpeza de Dados**

In [5]:
# Remover linhas com valores NaN
dados = dados.dropna()

# Substituir valores nulos por zero
dados['Filhos'] = dados['Filhos'].fillna(0)
dados['Filhos'] = dados['Filhos'].astype(int)
dados['Idade'] = dados['Idade'].fillna(0)
dados['Idade'] = dados['Idade'].astype(int)
dados['Encargos'] = dados['Encargos'].fillna(0)

dados['Gênero'] = dados['Gênero'].str.replace('nan', 'Não informado')

**3. Pré-processamento**

In [6]:
# Dividir os dados em features (X) e target (y)
X = dados.drop('Encargos', axis=1)  # Coluna alvo
y = dados['Encargos']

# Codificar variáveis categóricas usando one-hot encoding
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X[['Gênero', 'Região']])

# Concatenar features codificadas com as restantes
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['Gênero', 'Região']), index=X.index)
X = pd.concat([X.drop(['Gênero', 'Região'], axis=1), X_encoded], axis=1)

# Codificar variável categórica 'IMC' com LabelEncoder
label_encoder = LabelEncoder()
X['Categoria IMC'] = label_encoder.fit_transform(X['Categoria IMC'])

# Substituir valores 'sim' e 'não' por 1 e 0
X['Fumante'] = X['Fumante'].map({'sim': 1, 'não': 0})

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

correlation_matrix = X_train.corr()
print('Visualizar a matriz de correlação:')
print(correlation_matrix)

Visualizar a matriz de correlação:
                     Idade       IMC    Filhos   Fumante  Categoria IMC  \
Idade             1.000000 -0.065254 -0.011473 -0.013718       0.148605   
IMC              -0.065254  1.000000  0.140891  0.122271      -0.376520   
Filhos           -0.011473  0.140891  1.000000  0.049235       0.015639   
Fumante          -0.013718  0.122271  0.049235  1.000000      -0.008721   
Categoria IMC     0.148605 -0.376520  0.015639 -0.008721       1.000000   
Região_numerica   0.049741  0.013453 -0.060445  0.144306      -0.045932   
Gênero_masculino -0.001930  0.121831 -0.028254  0.150263      -0.091530   
Região_noroeste   0.149826 -0.025456 -0.011149  0.111067      -0.000852   
Região_sudeste   -0.025631  0.061523  0.101997  0.101794       0.061406   
Região_sudoeste   0.018472 -0.057799  0.018140 -0.175890       0.025068   

                  Região_numerica  Gênero_masculino  Região_noroeste  \
Idade                    0.049741         -0.001930         0.14982

**4. Treinamento do Modelo**

In [8]:
# Adicionando mais modelos ao dicionário de modelos
models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'Linear Regression': LinearRegression(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor()
}

# Treinamento e avaliação dos modelos
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Imprime uma linha separadora com base na largura do terminal
    print('-' * terminal_width)
    print(f'{name} MSE: {round(mse,2)}')
    # Calcular acurácia média usando validação cruzada
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f'Acurácia média com validação cruzada para {name}: {round(scores.mean(),2)}')
    
    # Calcula e exibe a importância das características para o modelo atual
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': model.feature_importances_})
        importance_df = importance_df.sort_values(by='Importância', ascending=False)
        print(f"Importância das características para {name}:")
        print(round(importance_df,2))
    elif isinstance(model, LinearRegression):
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Coeficiente': model.coef_})
        importance_df = importance_df.sort_values(by='Coeficiente', ascending=False)
        print(f"Coeficientes das características para {name}:")
        print(round(importance_df,2))
    elif isinstance(model, SVR):
        support_indices = model.support_
        support_features = X_train.iloc[support_indices]
        feature_importance_svr = support_features.mean(axis=0)
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': feature_importance_svr})
        importance_df = importance_df.sort_values(by='Importância', ascending=False)
        print(f"Importância das características para {name}:")
        print(round(importance_df,2))
    else:
        print(f"Não é possível calcular a importância das características para o modelo {name}")

--------------------------------------------------------------------------------------------------------------------------------------------------------------
Random Forest Regressor MSE: 230553154.6
Acurácia média com validação cruzada para Random Forest Regressor: -0.17
Importância das características para Random Forest Regressor:
            Colunas  Importância
1               IMC         0.37
0             Idade         0.31
2            Filhos         0.11
5   Região_numerica         0.06
3           Fumante         0.04
6  Gênero_masculino         0.03
8    Região_sudeste         0.02
4     Categoria IMC         0.02
9   Região_sudoeste         0.02
7   Região_noroeste         0.01
--------------------------------------------------------------------------------------------------------------------------------------------------------------
Linear Regression MSE: 212789125.79
Acurácia média com validação cruzada para Linear Regression: -0.13
Coeficientes das características para Li

**5. Comparação de Modelos**

In [9]:
# Seleção de modelo com base na menor pontuação de validação cruzada
best_model_name = min(models, key=lambda model: cross_val_score(models[model], X_train, y_train, cv=5).mean())

**6. Seleção de Modelo**

In [10]:
best_model = models[best_model_name]
print(f'Best Model: {best_model_name}')

Best Model: Decision Tree Regressor


**7. Otimização de Modelo**

In [11]:
# Otimização de hiperparâmetros com Optuna (exemplo com RandomForestRegressor)
def objective(trial):
    if best_model_name == 'Random Forest Regressor':
        n_estimators = trial.suggest_int('n_estimators', 10, 100)
        max_depth = trial.suggest_int('max_depth', 2, 32)
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    elif best_model_name == 'Decision Tree Regressor':
        max_depth = trial.suggest_int('max_depth', 2, 32)
        model = DecisionTreeRegressor(max_depth=max_depth)
    else:
        model = models[best_model_name]  # Use the default hyperparameters for other models
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=3, scoring='neg_mean_squared_error').mean()
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-04-09 10:19:43,545] A new study created in memory with name: no-name-b5ad42aa-7912-4815-b9ec-99013d144b88
[I 2024-04-09 10:19:46,078] Trial 0 finished with value: -234842498.08987257 and parameters: {'max_depth': 15}. Best is trial 0 with value: -234842498.08987257.
[I 2024-04-09 10:19:47,914] Trial 1 finished with value: -231221879.11640206 and parameters: {'max_depth': 20}. Best is trial 0 with value: -234842498.08987257.
[I 2024-04-09 10:19:49,847] Trial 2 finished with value: -232869799.50233245 and parameters: {'max_depth': 26}. Best is trial 0 with value: -234842498.08987257.
[I 2024-04-09 10:19:49,877] Trial 3 finished with value: -223799791.24848914 and parameters: {'max_depth': 26}. Best is trial 0 with value: -234842498.08987257.
[I 2024-04-09 10:19:49,905] Trial 4 finished with value: -230160666.47983027 and parameters: {'max_depth': 27}. Best is trial 0 with value: -234842498.08987257.
[I 2024-04-09 10:19:49,932] Trial 5 finished with value: -242596824.3415103 and p

In [12]:
best_params = study.best_params
best_model.set_params(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Optimized {best_model_name} MSE: {round(mse,2)}')

Optimized Decision Tree Regressor MSE: 289055334.46


In [13]:
# Obter a importância das características apenas para modelos que suportam esse atributo
if hasattr(best_model, 'feature_importances_'):
    # Obter a importância das características do modelo
    feature_importances = best_model.feature_importances_
    
    # Criar um DataFrame para visualizar a importância das características
    importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': feature_importances})

    # Ordenar as características por importância
    importance_df = importance_df.sort_values(by='Importância', ascending=False)
    
    print("\nQuais características individuais têm maior impacto nos custos médicos cobrados pelo seguro de saúde?")
    print(round(importance_df,2))

    # Selecionar as características mais importantes (por exemplo, as 10 mais importantes)
    top_features = importance_df.head(10)['Colunas'].tolist()  # Ajuste o número conforme necessário
    
    # Calcular a matriz de correlação apenas para as características mais importantes
    correlation_matrix = X_train[top_features].corr()
    
    # Visualizar a matriz de correlação
    print("\nExiste alguma correlação entre certas características (por exemplo, idade, IMC) e os custos médicos?")
    print(correlation_matrix)
else:
    print("O modelo selecionado não suporta o cálculo de importância das características.")


Quais características individuais têm maior impacto nos custos médicos cobrados pelo seguro de saúde?
            Colunas  Importância
1               IMC         0.51
0             Idade         0.28
3           Fumante         0.07
8    Região_sudeste         0.05
5   Região_numerica         0.03
2            Filhos         0.03
9   Região_sudoeste         0.01
6  Gênero_masculino         0.01
7   Região_noroeste         0.01
4     Categoria IMC         0.00

Existe alguma correlação entre certas características (por exemplo, idade, IMC) e os custos médicos?
                       IMC     Idade   Fumante  Região_sudeste  \
IMC               1.000000 -0.065254  0.122271        0.061523   
Idade            -0.065254  1.000000 -0.013718       -0.025631   
Fumante           0.122271 -0.013718  1.000000        0.101794   
Região_sudeste    0.061523 -0.025631  0.101794        1.000000   
Região_numerica   0.013453  0.049741  0.144306       -0.210525   
Filhos            0.140891 -0.011473

In [14]:
# Definição de dados futuros - 100 linhas
dados_futuros = pd.DataFrame(np.random.rand(100, len(X_train.columns)), columns=X_train.columns)
print(dados_futuros.head())

# Salvar os dados randômicos futuros
dados_futuros.to_csv("../planilhas/dados_futuros.csv", index=False, encoding='latin1')

encargos_futuros = prever_encargos_futuros(best_model, dados_futuros)
print("\nModelo treinado para fazer previsões dos encargos futuros:")
print(encargos_futuros)

grupos_risco = segmentacao_de_risco(best_model, dados_futuros)
print("\nGrupos de indivíduos com diferentes níveis de risco:")
print(grupos_risco)

custos_previstos_alterados = analise_de_sensibilidade(best_model, dados_futuros, 'Idade', novo_valor=40)
print("\nAnálises de sensibilidade para entender o impacto das mudanças nas características dos segurados nos encargos previstos:")
print(custos_previstos_alterados)

recursos_otimizados = otimizacao_de_recursos(encargos_futuros)
print("\nUtilizando as previsões dos encargos para otimizar a alocação de recursos:")
print(recursos_otimizados)

planos_estrategicos = planejamento_estrategico(best_model, dados_futuros)
print("\nUtilizando as informações obtidas com o modelo para desenvolver planos estratégicos:")
print(planos_estrategicos)

      Idade       IMC    Filhos   Fumante  Categoria IMC  Região_numerica  \
0  0.007073  0.323497  0.008475  0.599269       0.802124         0.961208   
1  0.684328  0.061355  0.819708  0.945553       0.841403         0.657175   
2  0.395672  0.892324  0.246464  0.710764       0.809546         0.861445   
3  0.842392  0.838597  0.700317  0.631468       0.837564         0.638636   
4  0.786210  0.208334  0.326173  0.988647       0.985408         0.589529   

   Gênero_masculino  Região_noroeste  Região_sudeste  Região_sudoeste  
0          0.536066         0.674708        0.642371         0.093649  
1          0.032764         0.890676        0.577488         0.032373  
2          0.585761         0.703344        0.245228         0.863693  
3          0.763157         0.582052        0.160226         0.135485  
4          0.508243         0.004471        0.836539         0.992748  

Modelo treinado para fazer previsões dos encargos futuros:
[51248.56, 51248.56, 51248.56, 51248.56, 5124